In [1]:
!nvidia-smi

Fri Nov 15 19:06:23 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:07:00.0 Off |                  Off |
| N/A   54C    P0   177W / 250W |  19355MiB / 24451MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:0F:00.0 Off |                  Off |
| N/A   28C    P8     9W / 250W |      0MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
import sys
sys.path.append('/Vol0/user/f.konokhov/psevdo-odometry/ ')

In [3]:
import __init_path__
import env

In [4]:
import functools
import argparse

import __init_path__
import env
from odometry.data_manager import GeneratorFactory
from odometry.evaluation import PredictCallback
from odometry.models import ModelFactory, construct_rigidity_model
from odometry.preprocessing.dataset_configs import get_config, DATASET_TYPES
from odometry.utils import str2bool
from odometry.models import construct_flexible_model

def train(dataset_root: str,
          dataset_type: str,
          predictions_dir: str = None,
          visuals_dir: str = None,
          period: int = 1,
          save_best_only: bool = True
          ) -> None:
    """
    This is script that shows how to use our small framework. You're welcome to modify it and experiment with new models.
    :param dataset_root:
    :param dataset_type:
    :param run_name:
    :param predictions_dir:
    :param visuals_dir:
    :param period:
    :param save_best_only:
    :return:
    """

    config = get_config(dataset_root, dataset_type)

    #  All parameters
    epochs = 50

    dataset = GeneratorFactory(
        csv_name='df.csv',
        dataset_root=dataset_root,
        train_trajectories=config['train_trajectories'],
        val_trajectories=config['val_trajectories'],
        test_trajectories=config['test_trajectories'],
        target_size=config['target_size'],
        x_col=['path_to_optical_flow'],
        image_col=['path_to_optical_flow'],
        load_mode=['flow_xy'],
        preprocess_mode=['flow_xy'],
        val_sampling_step=2,
        test_sampling_step=2,
        cached_images={}
    )

    construct_graph_fn = construct_flexible_model
    model_factory = ModelFactory(
        construct_graph_fn,
        input_shapes=dataset.input_shapes,
        lr=0.001,
        loss='mae',
        scale_rotation=50
    )
    model = model_factory.construct()

    train_generator = dataset.get_train_generator()
    val_generator = dataset.get_val_generator()
    val_steps = len(val_generator) if val_generator else None
    callback = PredictCallback(model,
                               dataset,
                               predictions_dir=predictions_dir,
                               visuals_dir=visuals_dir,
                               period=period,
                               save_best_only=save_best_only)

    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_steps,
        shuffle=True,
        callbacks=[callback]
    )

Using TensorFlow backend.
W1115 19:07:00.613363 47619418545792 deprecation.py:323] From /Vol0/user/f.konokhov/miniconda3/envs/odometry/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
train('/dbstore/datasets/Odometry_team/KITTI_odometry_2012', 'kitti_4/6',
      predictions_dir='/Vol0/user/f.konokhov/psevdo-odometry/mde',visuals_dir='/Vol0/user/f.konokhov/psevdo-odometry/mde')

100%|██████████| 6/6 [00:00<00:00, 51.57it/s]


Set empty cache
Set empty cache
Set empty cache
/Vol0/user/f.konokhov/psevdo-odometry/mde
Epoch 1/50
117/117 [==============================] - 127s 1s/step - loss: 0.7141 - euler_x_loss: 0.0024 - euler_y_loss: 0.0053 - euler_z_loss: 0.0024 - t_x_loss: 0.0139 - t_y_loss: 0.0108 - t_z_loss: 0.1827 - euler_x_rmse: 0.0024 - euler_y_rmse: 0.0053 - euler_z_rmse: 0.0024 - t_x_rmse: 0.0139 - t_y_rmse: 0.0108 - t_z_rmse: 0.1827 - val_loss: 0.4037 - val_euler_x_loss: 0.0025 - val_euler_y_loss: 0.0013 - val_euler_z_loss: 0.0019 - val_t_x_loss: 0.0082 - val_t_y_loss: 0.0064 - val_t_z_loss: 0.1035 - val_euler_x_rmse: 0.0025 - val_euler_y_rmse: 0.0014 - val_euler_z_rmse: 0.0019 - val_t_x_rmse: 0.0082 - val_t_y_rmse: 0.0064 - val_t_z_rmse: 0.1035
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/00
00
sssssssssssssssssssss 00
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 001_train:0.717003_val:0.403664 train 00.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/001_train:0.717003_val:

/Vol0/user/f.konokhov/miniconda3/envs/odometry/lib/python3.6/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/02
02
sssssssssssssssssssss 02
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 001_train:0.717003_val:0.403664 train 02.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/001_train:0.717003_val:0.403664/train/02.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/08
08
sssssssssssssssssssss 08
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 001_train:0.717003_val:0.403664 train 08.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/001_train:0.717003_val:0.403664/train/08.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/09
09
sssssssssssssssssssss 09
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 001_train:0.717003_val:0.403664 train 09.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/001_train:0.717003_val:0.403664/train/09.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/03
03
sssssssssssssssssssss 03
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 001_train:0.717003_v

trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/03
03
sssssssssssssssssssss 03
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 004_train:0.224830_val:0.165316 val 03.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/004_train:0.224830_val:0.165316/val/03.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/04
04
sssssssssssssssssssss 04
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 004_train:0.224830_val:0.165316 val 04.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/004_train:0.224830_val:0.165316/val/04.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/05
05
sssssssssssssssssssss 05
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 004_train:0.224830_val:0.165316 val 05.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/004_train:0.224830_val:0.165316/val/05.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/06
06
sssssssssssssssssssss 06
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 004_train:0.224830_val:0.165316 

trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/03
03
sssssssssssssssssssss 03
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 012_train:0.149247_val:0.160128 val 03.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/012_train:0.149247_val:0.160128/val/03.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/04
04
sssssssssssssssssssss 04
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 012_train:0.149247_val:0.160128 val 04.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/012_train:0.149247_val:0.160128/val/04.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/05
05
sssssssssssssssssssss 05
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 012_train:0.149247_val:0.160128 val 05.csv
mda /Vol0/user/f.konokhov/psevdo-odometry/mde/012_train:0.149247_val:0.160128/val/05.csv
trajectory_id /dbstore/datasets/Odometry_team/KITTI_odometry_2012/06
06
sssssssssssssssssssss 06
wtf /Vol0/user/f.konokhov/psevdo-odometry/mde 012_train:0.149247_val:0.160128 

E1115 19:27:10.202548 47619418545792 ultratb.py:149] Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
import sys

In [ ]:
train('/dbstore/datasets/Odometry_team/KITTI_odometry_2012', 'kitti_4/6', 'mde','mda')

In [ ]:
'/dbstore/datasets/Odometry_team/KITTI_odometry_2012/'

In [ ]:
from tensorflow.compat.v1 import ConfigProto

In [ ]:
from odometry.models.layers import concat